In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from torch.utils.data import Dataset
import pickle
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import torchsample as ts
from torchsample.transforms import RandomRotate
%matplotlib inline

## DATASET
Class for loading data set from the pickle format.

Label encoding was performed as classes given were 2,3,0 and 6.

Data augmentation was applied via few torch vision transformations on the data set

In [ ]:
class ImageDataset(Dataset):
    
    def __init__(self, csv_files,transform):
        super().__init__()
        self.csv_files=csv_files
        with open(csv_files[0], 'rb') as f:
            X = np.array(pickle.load(f),dtype=np.float32)
        
        if csv_files[1] is not None:
            with open(csv_files[1], 'rb') as g:
                y = np.array(pickle.load(g))
            le=LabelEncoder()
            y_label=le.fit_transform(y)
            self.y=y_label
            self.le =le 
            
        self.transform=transform
        
        self.X= X.reshape(-1,1,28,28)
#         self.X= X.reshape(-1,1,28,28)
#         print(np.mean(np.mean(self.X,axis=0)))
              
                
    def get_class_count(self):
        return len(self.le.classes_)
    
    def get_input_dim(self):
        return self.X.shape[1]
    
    def get_class_labels(self,y_label):
        le=self.le
        return le.inverse_transform(y_label)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
         z=self.X[idx]
         #print(z.shape)
         z=np.reshape(z, (28,28))
         img_name = Image.fromarray(z)
#         plt.imshow(img_name)
         sample=self.X[idx]
        
         if self.transform is not None:
            sample = self.transform(img_name)
#             #plt.show()
            sample= sample.reshape(1,28,28)         
            return sample,self.y[idx]
         
         return sample

## Data Loading and Pre-Processing

Data Augmentation was performed using the following torch vision transforms:

(1) RandomHorizontalFlip

(2) RandomAffine of degree 30 

(3) Normalization of the pixel values of the images.

Data was loaded as training, validation and augment sets with normalization performed for training data and transforms performed for augment data and normalization performed for validation data.

Use of SubsetRandomSampler to split into train and validation.


In [ ]:
valid_size=0.3
shuffle=True
valid_transform = transforms.Compose([transforms.ToTensor(),  transforms.Normalize(mean=[82.34472], std=[255]),])

augment_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomAffine(30), transforms.ToTensor(),  transforms.Normalize(mean=[82.34472], std=[255]),])
train_transform = transforms.Compose([ transforms.ToTensor(),  transforms.Normalize(mean=[82.34472], std=[255]),])

train_dataset = ImageDataset(['train_image.pkl','train_label.pkl'],train_transform)
valid_dataset = ImageDataset(['train_image.pkl','train_label.pkl'],valid_transform)
augment_dataset = ImageDataset(['train_image.pkl','train_label.pkl'],augment_transform)
#test_dataset = ImageDataset(['test_image.pkl', None],transforms.ToTensor())
batch_size=512
num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
if shuffle:
        np.random.seed(42)
        np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)
augment_loader = torch.utils.data.DataLoader(augment_dataset, batch_size=batch_size, sampler=valid_sampler)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))

In [ ]:
import torch.nn as nn

n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = 1000

## Model - Convolutional Neural Network
CNN Network of 2 Convolution Layers followed by two fully connected layers and a layer of dropout

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.batch1=nn.BatchNorm2d(32)
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.batch2=nn.BatchNorm2d(64)
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
#         self.relu3 = nn.ReLU()
#         self.batch3=nn.BatchNorm2d(64)
#         # Max pool 3
#         self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(in_features=64 * 4 * 4, out_features=60)
        self.droput = nn.Dropout(p=0.5)
        self.outer = nn.Linear(in_features=60, out_features=4)
        
    def forward(self,x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.batch1(out)
        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.batch2(out)
        # Max pool 2 
        out = self.maxpool2(out)
        
#         # Convolution 3 
#         out = self.conv3(out)
#         out = self.relu3(out)
#         out = self.batch3(out)
#         # Max pool 3 
#         out = self.maxpool3(out)

        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.fc2(out)
        out = self.droput(out)
        out = self.outer(out)
        return out


Adam Optimizer was used as learning algorithm and Cross Entropy was used as loss function

In [ ]:
model = Network().cuda(1)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
iter = 0
for epoch in range(num_epochs):
    model.train()
    training_loss_list = []
    testing_loss_list = []
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Load images as tensors with gradient accumulation abilities
        images = images.requires_grad_().cuda(1)
        labels =labels.cuda(1)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        training_loss_list.append(loss.item())
    for i, (images, labels) in enumerate(augment_loader):
    # Load images as tensors with gradient accumulation abilities
        images = images.requires_grad_().cuda(1)
        labels =labels.cuda(1)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        training_loss_list.append(loss.item())
    training_loss = sum(training_loss_list)/len(training_loss_list)    

    model.eval()    
    if(epoch%2==0):
            # Calculate Accuracy         
        correct = 0
        total = 0
            # Iterate through test dataset
        losses = []
        for images, labels in validation_loader:
            # Load images to tensors with gradient accumulation abilities
            images = images.requires_grad_().cuda(1)
            labels = labels.cuda(1)

            # Forward pass only to get logits/output
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            testing_loss_list.append(loss.item())
            # Get predictions from the maximum value
            _, predicted = torch.max(outputs.data, 1)

            # Total number of labels
            total += labels.size(0)

            # Total correct predictions
            correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
        test_loss = sum(testing_loss_list)/len(testing_loss_list)
        torch.save(model.state_dict(),str(epoch))
        print('Epoch: {}. Training_Loss: {}.Testing_Loss:{}. Accuracy: {}'.format(epoch, training_loss,test_loss ,accuracy))

In [ ]:
with open('test_image.pkl', 'rb') as f:
    X = np.array(pickle.load(f),dtype=np.float32)
y = F.softmax(model(images))
_, predicted = torch.max(y.data, 1)
print(predicted)
# access Variable's tensor, copy back to CPU, convert to numpy
arr = predicted.data.cpu().numpy()
arr=np.array(arr,dtype='uint8')
final=train_dataset.get_class_labels(arr)
# write CSV
np.savetxt('output.csv', final)